# Imports

In [3]:
from collections import Counter
import community as community_louvain
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network

# Functions definition

In [4]:
def get_ngrams(sequence, n):
    """Generates n-grams from a list sequence efficiently."""
    if n == 0:
        return []
    # Using zip to create sliding windows is efficient in Python
    return zip(*[sequence[i:] for i in range(n)])


def find_significant_motifs(flights_data, k, z_threshold=1.96, top_percentile=0.05):
    """
    Identifies statistically significant k-motifs.
    
    Args:
        flights_data (list of lists): Each inner list is a sequence of flight phase integers.
        k (int): The length of the motif to analyze (e.g., 3).
        z_threshold (float): Z-score cutoff (1.96 for 95% confidence).
        
    Returns:
        pd.DataFrame: Table of motifs with Obs/Exp probabilities and Z-scores.
    """
    # 1. Count frequencies for k, k-1, and k-2 patterns
    # We use a single pass over the data to populate all counters
    counts_k = Counter()      # Counts for x_1...x_k
    counts_prefix = Counter() # Counts for x_1...x_{k-1}
    counts_overlap = Counter() # Counts for x_2...x_{k-1} (the middle part)
    
    for flight in flights_data:
        # flight is a list of ints, e.g., [10, 20, 30, 40]
        if len(flight) < k:
            continue

        # Count k-grams (x_1...x_k)
        ngrams = list(get_ngrams(flight, k))
        counts_k.update(ngrams)
            
        # Note: If k=2, p_exp = p_obs (Eq. 5)
        if k > 2:
            # Count (k-1)-grams (used for prefix and suffix)
            ngrams_minus_1 = list(get_ngrams(flight, k - 1))
            counts_prefix.update(ngrams_minus_1)

            # Count (k-2)-grams (used for overlap)
            ngrams_minus_2 = list(get_ngrams(flight, k - 2))
            counts_overlap.update(ngrams_minus_2)
    
    # Track total number of substrings of each length for probability normalization
    total_k = counts_k.total()
    total_prefix = counts_prefix.total()
    total_overlap = counts_overlap.total()
    
    # 2. Calculate Probabilities and Z-scores
    results = []

    if k == 2:
        n = int(top_percentile*total_k)
        for motif, count in counts_k.most_common(n):
            results.append({
                "motif": motif,
                "count": count,
                "p_obs": count/total_k,
                "p_exp": count/total_k,
                "z_score": None
            })
        
        # 3. Format Output
        df = pd.DataFrame(results)
        if not df.empty:
            df = df.sort_values(by="count", ascending=False)
        return df
            
    
    for motif, count in counts_k.items():
        
        # Define parts of the motif
        # motif is a tuple like (A, B, C)
        prefix = motif[:-1]      # (A, B)
        suffix = motif[1:]       # (B, C)
        overlap = motif[1:-1]    # (B)

        # Observed Probability: p_obs(ABC)
        p_obs = count / total_k
        
        # Expected Probability calculation 
        # p_exp(ABC) = p_obs(AB) * p_obs(BC) / p_obs(B)
        prob_prefix = counts_prefix[prefix] / total_prefix
        prob_suffix = counts_prefix[suffix] / total_prefix

        prob_overlap = counts_overlap[overlap] / total_overlap
        if prob_overlap == 0:
            continue # Avoid division by zero
        p_exp = (prob_prefix * prob_suffix) / prob_overlap
            
        # Calculate Z-score
        # Standard Deviation for binomial distribution approx: sqrt(N * p * (1-p))
        
        expected_count = p_exp * total_k
        sigma = math.sqrt(total_k * p_exp * (1 - p_exp))
        
        if sigma == 0:
            z_score = 0 # p_exp is too small (prevent floating point issues)
        else:
            z_score = (count - expected_count) / sigma

        if z_score > z_threshold:
            results.append({
                "motif": motif,
                "count": count,
                "p_obs": p_obs,
                "p_exp": p_exp,
                "z_score": z_score
            })
            
    # 3. Format Output
    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values(by="z_score", ascending=False)
    
    return df


def build_motif_adjacency_matrix(flights_data, significant_motifs_df, k, z_threshold=1.96):
    """
    Constructs the weighted adjacency matrix for the k-motif network.
    
    Args:
        flights_data (list of lists): The original flight sequences.
        significant_motifs_df (pd.DataFrame): Output from find_significant_motifs().
        k (int): Motif length.
        z_threshold (float): Z-score filter for edges.
        
    Returns:
        pd.DataFrame: Adjacency matrix (Z-scores) where rows=Source, cols=Target.
    """
    
    # 1. Setup Nodes and Indices
    # Filter only the motifs that were deemed significant
    valid_motifs = significant_motifs_df['motif'].tolist()
    motif_to_idx = {motif: i for i, motif in enumerate(valid_motifs)}
    n_motifs = len(valid_motifs)
    
    # Extract probabilities p(X) for Eq. 6
    # Create a mapping of motif -> p_obs
    motif_probs = significant_motifs_df.set_index('motif')['p_obs'].to_dict()
    
    # 2. Pre-calculate the Normalization Constant (Sum term from Eq. 6)
    # Only sequences with length >= 2k can physically contain two non-overlapping motifs
    normalization_sum = 0
    for flight in flights_data:
        l_s = len(flight)
        if l_s >= 2 * k:
            term = (l_s - 2 * k + 1) * (l_s - 2 * k + 2)
            normalization_sum += term
            
    # 3. Count Observed Co-occurrences
    # Matrix to store N_obs(X -> Y)
    observed_matrix = np.zeros((n_motifs, n_motifs))
    
    # We must scan flights again to find where motifs land
    # Optimization: Convert flights to lists of motif indices
    for flight in flights_data:
        if len(flight) < 2 * k:
            continue
            
        # Find all instances of significant motifs in this flight
        # Store as (start_index, motif_id)
        instances = []
        
        # Sliding window to find motifs
        # Note: This is O(L * k), can be optimized but sufficient for typical data
        for i in range(len(flight) - k + 1):
            segment = tuple(flight[i : i+k])
            if segment in motif_to_idx:
                instances.append((i, motif_to_idx[segment]))
        
        # Check pairs for valid temporal ordering (non-overlapping)
        # Y must start at least k steps after X starts
        for i in range(len(instances)):
            start_x, id_x = instances[i]
            
            # Look ahead for Y
            for j in range(i + 1, len(instances)):
                start_y, id_y = instances[j]
                
                if start_y >= start_x + k:
                    # Valid non-overlapping sequence X -> Y found
                    observed_matrix[id_x, id_y] += 1
    
    # 4. Calculate Expected Counts and Z-scores
    # Eq 6: Exp(X, Y) = 0.5 * p(X) * p(Y) * normalization_sum
    
    # Create vectors for p(X) and p(Y) aligned with the matrix indices
    probs_vector = np.array([motif_probs[m] for m in valid_motifs])
    
    # Outer product to get p(X)*p(Y) matrix efficiently
    prob_product_matrix = np.outer(probs_vector, probs_vector)
    
    expected_matrix = 0.5 * prob_product_matrix * normalization_sum
    
    # Calculate Z-scores
    # Avoid division by zero and sqrt of negative numbers
    z_score_matrix = np.zeros((n_motifs, n_motifs))
    
    with np.errstate(divide='ignore', invalid='ignore'):
        # Using Poisson approximation for standard deviation: sigma = sqrt(expected)
        # For stricter binomial sigma, we'd need Total Possible Pairs, 
        # but sqrt(exp) is standard for rare network events.
        sigma_matrix = np.sqrt(expected_matrix)
        
        raw_z = (observed_matrix - expected_matrix) / sigma_matrix
        
        # Clean up NaNs/Infs (where expected is 0)
        z_score_matrix = np.nan_to_num(raw_z)

    # 5. Format as DataFrame
    adj_df = pd.DataFrame(z_score_matrix, index=valid_motifs, columns=valid_motifs)
    
    # Apply Threshold (paper mentions "statistically significant co-occurrences")
    # We zero out insignificant links to make the network sparse
    adj_df[adj_df < z_threshold] = 0
    
    return adj_df


def plot_static_graph(G):
    plt.figure(figsize=(12, 12))
    
    # 1. Calculate Layout (Spring layout positions nodes based on connections)
    pos = nx.spring_layout(G, k=0.5, seed=42)  # k regulates distance between nodes
    
    # 2. Extract Weights for styling
    weights = [G[u][v]['weight'] for u, v in G.edges()]
    
    # Normalize weights for visualization (e.g., thickness between 0.5 and 4.5)
    # Avoid division by zero if all weights are same
    if max(weights) > min(weights):
        width = [(w - min(weights))/(max(weights)-min(weights)) * 4 + 0.5 for w in weights]
    else:
        width = [1.0 for _ in weights]

    # 3. Draw the Network
    # Nodes
    nx.draw_networkx_nodes(G, pos, node_size=700, node_color='lightblue')
    
    # Edges (Width varies by Z-score)
    nx.draw_networkx_edges(G, pos, width=width, edge_color='gray', 
                           arrowstyle='->', arrowsize=20)
    
    # Labels (Motif names)
    nx.draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif")
    
    plt.title("Flight Phase K-Motif Network (Weighted by Z-Score)")
    plt.axis('off')
    plt.show()


def plot_interactive_graph(G, filename="motif_network.html"):
    # Initialize PyVis network
    net = Network(height="750px", width="100%", notebook=True, directed=True)
    
    # Optional: Detect Communities (Louvain method) to color nodes
    try:
        partition = community_louvain.best_partition(G.to_undirected())
        # Add partition info to node attributes for coloring
        for node, group_id in partition.items():
            G.nodes[node]['group'] = group_id
    except ImportError:
        print("Community detection skipped (install 'python-louvain' for colors)")

    # Convert NetworkX graph to PyVis
    net.from_nx(G)
    
    # Customizing physics for better separation (optional)
    net.set_options("""
    var options = {
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "centralGravity": 0.01,
          "springLength": 100,
          "springConstant": 0.08
        },
        "maxVelocity": 50,
        "solver": "forceAtlas2Based",
        "timestep": 0.35,
        "stabilization": { "enabled": true }
      }
    }
    """)
    
    # Save and show
    net.show(filename)
    return filename

# Run

In [6]:
df = pd.read_csv("PIE_data_with_context.csv", sep=";", header=0, index_col=0)
df = df.sort_values(by=['F_SESSION', 'F_START_FRAME'], ascending=[True, True])
df.head()

,F_SESSION,F_START_FRAME,F_END_FRAME,F_DURATION,FIRST_WORD_INDEX,SECOND_WORD_INDEX,THIRD_WORD_INDEX,session,k_aircraft,k_operator,k_mission
879696,3130311,332,375,00:00:22.000,3.0,10.0,285.0,3130311,46852,20,-2
879697,3130311,376,441,00:00:33.000,3.0,75.0,281.0,3130311,46852,20,-2
879698,3130311,442,741,00:02:30.000,3.0,10.0,8.0,3130311,46852,20,-2
879699,3130311,742,760,00:00:09.500,3.0,28.0,39.0,3130311,46852,20,-2
879700,3130311,761,764,00:00:02.000,3.0,10.0,8.0,3130311,46852,20,-2


In [ ]:
# --- Example Usage ---
# Assuming you load your CSV into a list of lists:
# flights = [[1, 2, 3, 4], [1, 2, 5], ...] 
# df_significant = find_significant_motifs(flights, k=3)
# print(df_significant.head())

# --- Usage Example ---
# adj_matrix = build_motif_adjacency_matrix(all_flights, df_significant, k=3)
# print(adj_matrix.shape)

# # 1. Create Directed Graph from Adjacency Matrix
# # The matrix rows are sources, columns are targets
# G = nx.from_pandas_adjacency(adj_df, create_using=nx.DiGraph)

# # 2. Remove edges with Weight = 0 (Insignificant links)
# # networkx creates edges for 0 values by default, which we don't want
# edges_to_remove = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] == 0]
# G.remove_edges_from(edges_to_remove)

# print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")